In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, DDPMScheduler
import math
import glob
import random
import json

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import os
import json
from tqdm import tqdm
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionPipeline
from transformers import CLIPTokenizer, CLIPTextModel

In [6]:
import torch
import random
import numpy as np

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
# Sinusoidal timestep embedding for diffusion steps
def get_timestep_embedding(timesteps, embedding_dim):
    half_dim = embedding_dim // 2
    emb = torch.exp(
        torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) *
        -(torch.log(torch.tensor(10000.0)) / half_dim)
    )
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # Handle odd embedding dimensions
        emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
    return emb

# Residual block with time and context embeddings
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_emb_dim, context_dim=None):
        super().__init__()
        self.norm1 = nn.GroupNorm(min(32, in_channels), in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.norm2 = nn.GroupNorm(min(32, out_channels), out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_mlp = nn.Linear(time_emb_dim, out_channels)
        self.context_proj = nn.Linear(context_dim, out_channels) if context_dim else None
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, t_emb, context=None):
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)

        # Add time embedding
        t_proj = self.time_mlp(t_emb)[:, :, None, None]
        h = h + t_proj

        # Add context embedding if available
        if self.context_proj is not None and context is not None:
            context_pooled = context.mean(dim=1)  # [batch, context_dim]
            context_proj = self.context_proj(context_pooled)[:, :, None, None]
            h = h + context_proj

        h = self.norm2(h)
        h = F.silu(h)
        h = self.conv2(h)

        return h + self.shortcut(x)

# Cross-attention to integrate text embeddings
class CrossAttention(nn.Module):
    def __init__(self, channels, context_dim):
        super().__init__()
        self.channels = channels
        self.query = nn.Linear(channels, channels)
        self.key = nn.Linear(context_dim, channels)
        self.value = nn.Linear(context_dim, channels)
        self.out = nn.Linear(channels, channels)
        self.norm = nn.LayerNorm(channels)

    def forward(self, x, context):
        if context is None:
            return x

        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm(x_flat)

        q = self.query(x_norm)  # [B, H*W, C]
        k = self.key(context)   # [B, seq_len, C]
        v = self.value(context) # [B, seq_len, C]

        scale = (C ** -0.5)
        attn_weights = torch.bmm(q, k.transpose(1, 2)) * scale
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_out = torch.bmm(attn_weights, v)
        attn_out = self.out(attn_out)

        attn_out = attn_out.reshape(B, H, W, C).permute(0, 3, 1, 2)
        return x + attn_out

# Self-attention block for image features
class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.qkv = nn.Conv2d(channels, channels * 3, 1)
        self.proj = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        qkv = self.qkv(h).reshape(B, 3, C, H * W)
        q, k, v = qkv[:, 0], qkv[:, 1], qkv[:, 2]

        scale = (C ** -0.5)
        attn = torch.bmm(q.transpose(1, 2), k) * scale
        attn = F.softmax(attn, dim=-1)

        out = torch.bmm(v, attn.transpose(1, 2))
        out = out.reshape(B, C, H, W)
        return self.proj(out) + x

# U-Net model updated for 256x256 latents
class UNetConditional(nn.Module):
    def __init__(self, in_channels=4, base_channels=128, context_dim=768):
        super().__init__()
        self.time_emb_dim = base_channels * 4
        from types import SimpleNamespace
        self.config = SimpleNamespace()
        self.config._diffusers_version = "0.34.0"
        self.config.in_channels = in_channels
        self.config.out_channels = in_channels
        self.config.sample_size = 256  # Updated for 256x256 latents
        self.config.layers_per_block = 2
        self.config.block_out_channels = [base_channels, base_channels * 2, base_channels * 4, base_channels * 8]
        self.config.attention_head_dim = 8
        self.config.cross_attention_dim = context_dim

        # Time embedding MLP
        self.time_mlp = nn.Sequential(
            nn.Linear(base_channels, self.time_emb_dim),
            nn.SiLU(),
            nn.Linear(self.time_emb_dim, self.time_emb_dim),
        )

        # Input projection
        self.input_conv = nn.Conv2d(in_channels, base_channels, 3, padding=1)

        # Encoder
        self.down1 = ResidualBlock(base_channels, base_channels * 2, self.time_emb_dim, context_dim)
        self.downsample1 = nn.Conv2d(base_channels * 2, base_channels * 2, 3, stride=2, padding=1)
        self.cross1 = CrossAttention(base_channels * 2, context_dim)

        self.down2 = ResidualBlock(base_channels * 2, base_channels * 4, self.time_emb_dim, context_dim)
        self.downsample2 = nn.Conv2d(base_channels * 4, base_channels * 4, 3, stride=2, padding=1)
        self.cross2 = CrossAttention(base_channels * 4, context_dim)

        self.down3 = ResidualBlock(base_channels * 4, base_channels * 8, self.time_emb_dim, context_dim)
        self.downsample3 = nn.Conv2d(base_channels * 8, base_channels * 8, 3, stride=2, padding=1)
        self.cross3 = CrossAttention(base_channels * 8, context_dim)

        # Middle
        self.middle1 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)
        self.middle_attn = AttentionBlock(base_channels * 8)
        self.middle2 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)

        # Decoder
        self.up3 = ResidualBlock(base_channels * 16, base_channels * 4, self.time_emb_dim, context_dim)
        self.upsample3 = nn.ConvTranspose2d(base_channels * 4, base_channels * 4, 4, stride=2, padding=1)
        self.cross_up3 = CrossAttention(base_channels * 4, context_dim)

        self.up2 = ResidualBlock(base_channels * 8, base_channels * 2, self.time_emb_dim, context_dim)
        self.upsample2 = nn.ConvTranspose2d(base_channels * 2, base_channels * 2, 4, stride=2, padding=1)
        self.cross_up2 = CrossAttention(base_channels * 2, context_dim)

        self.up1 = ResidualBlock(base_channels * 4, base_channels, self.time_emb_dim, context_dim)
        self.upsample1 = nn.ConvTranspose2d(base_channels, base_channels, 4, stride=2, padding=1)

        # Output
        self.output_conv = nn.Sequential(
            nn.GroupNorm(min(32, base_channels), base_channels),
            nn.SiLU(),
            nn.Conv2d(base_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t, context, cfg_scale=1.0):
        t_emb = get_timestep_embedding(t, self.time_emb_dim // 4)
        t_emb = self.time_mlp(t_emb)

        def denoise(x, t_emb, context):
            h = self.input_conv(x)

            # Encoder
            h1 = self.down1(h, t_emb, context)
            h1_cross = self.cross1(h1, context)
            h1_down = self.downsample1(h1_cross)

            h2 = self.down2(h1_down, t_emb, context)
            h2_cross = self.cross2(h2, context)
            h2_down = self.downsample2(h2_cross)

            h3 = self.down3(h2_down, t_emb, context)
            h3_cross = self.cross3(h3, context)
            h3_down = self.downsample3(h3_cross)

            # Middle
            h_mid = self.middle1(h3_down, t_emb, context)
            h_mid = self.middle_attn(h_mid)
            h_mid = self.middle2(h_mid, t_emb, context)

            # Decoder
            h3_cross_resized = F.interpolate(h3_cross, size=h_mid.shape[-2:], mode='nearest')
            h = self.up3(torch.cat([h_mid, h3_cross_resized], dim=1), t_emb, context)
            h = self.upsample3(h)
            h = self.cross_up3(h, context)

            h2_cross_resized = F.interpolate(h2_cross, size=h.shape[-2:], mode='nearest')
            h = self.up2(torch.cat([h, h2_cross_resized], dim=1), t_emb, context)
            h = self.upsample2(h)
            h = self.cross_up2(h, context)

            h1_cross_resized = F.interpolate(h1_cross, size=h.shape[-2:], mode='nearest')
            h = self.up1(torch.cat([h, h1_cross_resized], dim=1), t_emb, context)
            h = self.upsample1(h)

            return self.output_conv(h)

        if cfg_scale == 1.0 or context is None:
            return denoise(x, t_emb, context)

        uncond = denoise(x, t_emb, context=None)
        cond = denoise(x, t_emb, context)
        return uncond + cfg_scale * (cond - uncond)

# Dataset for pixel art images
class PixelArtDataset(Dataset):
    def __init__(self, images_folder, captions, image_size=256):
        self.images_folder = images_folder
        # Adjusting to handle the provided JSON format
        self.captions = [(k, v) for k, v in captions.items() if isinstance(v, str)]
        self.image_size = image_size
        self.transform = T.Compose([
            T.Resize((image_size, image_size), interpolation=T.InterpolationMode.LANCZOS),
            T.ToTensor(),
            T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        print(f"Initialized dataset with {len(self.captions)} valid image-caption pairs")


    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        max_retries = 5
        for retry in range(max_retries):
            try:
                current_idx = (idx + retry) % len(self.captions)
                image_name, prompt = self.captions[current_idx]
                image_path = os.path.join(self.images_folder, image_name)

                if not os.path.exists(image_path):
                    print(f"Warning: Image not found: {image_path}")
                    continue

                image = Image.open(image_path).convert("RGB")
                image_tensor = self.transform(image)

                return {
                    "image": image_tensor,
                    "prompt": prompt
                }
            except Exception as e:
                print(f"Error loading item {current_idx}: {e}")
                if retry == max_retries - 1:
                    return {
                        "image": torch.zeros(3, self.image_size, self.image_size),
                        "prompt": "a pixel art image"
                    }
                continue

# Collate function for batching
def collate_fn(batch):
    images = torch.stack([item['image'] for item in batch])
    prompts = [item['prompt'] for item in batch]
    return {'image': images, 'prompt': prompts}

# Generate test images with the trained model
def generate_test_images(vae, text_encoder, tokenizer, unet, scheduler, device="cuda", num_inference_steps=20, guidance_scale=7.5):
    print(f"Using device: {device}")

    test_prompts = [
        "a pixel art cat",
        "a pixel art house",
        "a pixel art tree",
        "james graham ballard, highrise, sustainability, octane render, highly detailed",
        "pixel art landscape"
    ]

    print("🎨 Generating test images directly...")

    for i, prompt in enumerate(test_prompts):
        print(f"Generating: {prompt}")

        try:
            with torch.no_grad():
                # Encode prompt
                inputs = tokenizer(
                    prompt,
                    padding="max_length",
                    truncation=True,
                    max_length=tokenizer.model_max_length,
                    return_tensors="pt"
                )
                text_embeddings = text_encoder(inputs.input_ids.to(device))[0]

                # Generate unconditional embeddings for classifier-free guidance
                if guidance_scale > 1.0:
                    uncond_inputs = tokenizer(
                        [""], # Use an empty string for unconditional generation
                        padding="max_length",
                        truncation=True,
                        max_length=tokenizer.model_max_length,
                        return_tensors="pt"
                    )
                    uncond_embeddings = text_encoder(uncond_inputs.input_ids.to(device))[0]
                    text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

                # Initial random latents (batch size 1, 4 channels, 32x32 for 256x256 output)
                latents = torch.randn(
                    1,
                    unet.config.in_channels,
                    unet.config.sample_size // 8, # VAE scaling
                    unet.config.sample_size // 8,
                    generator=torch.Generator(device=device).manual_seed(42 + i) # Set seed for reproducibility
                ).to(device)

                # Set timesteps
                scheduler.set_timesteps(num_inference_steps, device=device)
                timesteps = scheduler.timesteps

                # Denoising loop
                for t in tqdm(timesteps, desc=f"Denoising {prompt}"):
                    # Expand latents for classifier-free guidance
                    latent_model_input = torch.cat([latents] * 2) if guidance_scale > 1.0 else latents
                    latent_model_input = scheduler.scale_model_input(latent_model_input, t)

                    # Predict noise
                    # Ensure timestep is on the same device as latents and unet
                    t_tensor = torch.tensor([t], device=latents.device)
                    if guidance_scale > 1.0:
                         # Pass the concatenated text embeddings
                        noise_pred = unet(latent_model_input, t_tensor.repeat(2), context=text_embeddings)
                        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                        noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
                    else:
                         noise_pred = unet(latent_model_input, t_tensor, context=text_embeddings)


                    # Compute the previous noisy sample x_t -> x_t-1
                    latents = scheduler.step(noise_pred, t, latents).prev_sample

                # Decode latents to image
                latents = 1 / vae.config.scaling_factor * latents # Rescale latents
                image = vae.decode(latents).sample

                # Post-processing
                image = (image / 2 + 0.5).clamp(0, 1) # Denormalize and clamp
                image = image.cpu().permute(0, 2, 3, 1).numpy()[0] # Move to CPU, permute, convert to numpy

                # Convert to PIL Image and save
                image = Image.fromarray((image * 255).astype(np.uint8))
                safe_filename = f"/content/generated_256_{i+1}_{prompt[:30].replace(' ', '_').replace(',', '').replace('.', '')}.png"
                image.save(safe_filename)
                print(f"✅ Saved: {safe_filename}")

        except Exception as e:
            print(f"❌ Error generating '{prompt}': {e}")
            print(f"Error type: {type(e).__name__}")
            # Continue with next prompt
            continue

# Main training function
def train_diffusion_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load dataset
    images_folder = 'kkm'
    captions_file = 'kahab.json'
    with open(captions_file, 'r') as f:
        captions = json.load(f)
    print(f"Loaded {len(captions)} image-caption pairs")

    dataset = PixelArtDataset(images_folder, captions, image_size=256)
    loader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_fn,
        drop_last=True
    )

    # Initialize models
    vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
    vae = vae.eval().requires_grad_(False).to(device)

    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = text_encoder.eval().requires_grad_(False).to(device)

    unet = UNetConditional(in_channels=4, base_channels=128, context_dim=768).to(device)

    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    optimizer = optim.AdamW(unet.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000, eta_min=1e-6)

    num_epochs = 600
    save_every = 100
    print("Starting training...")

    for epoch in range(num_epochs):
        epoch_loss = 0
        successful_batches = 0
        unet.train()
        progress_bar = tqdm(loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for batch_idx, batch in enumerate(progress_bar):
            try:
                images = batch['image'].to(device, non_blocking=True)
                prompts = batch['prompt']

                if images.shape[0] == 0:
                    print(f"Empty batch {batch_idx}, skipping...")
                    continue

                inputs = tokenizer(
                    prompts,
                    padding="max_length",
                    truncation=True,
                    max_length=77,
                    return_tensors="pt"
                )
                inputs = {k: v.to(device, non_blocking=True) for k, v in inputs.items()}

                with torch.no_grad():
                    text_embeddings = text_encoder(**inputs).last_hidden_state
                    latents = vae.encode(images).latent_dist.sample() * 0.18215

                noise = torch.randn_like(latents)
                timesteps = torch.randint(
                    0, noise_scheduler.config.num_train_timesteps,
                    (latents.size(0),), device=device
                ).long()

                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
                pred_noise = unet(noisy_latents, timesteps, text_embeddings)

                if pred_noise.shape != noise.shape:
                    print(f"Shape mismatch: pred_noise {pred_noise.shape} vs noise {noise.shape}")
                    continue

                loss = F.mse_loss(pred_noise, noise)
                if torch.isnan(loss):
                    print(f"NaN loss detected in batch {batch_idx}, skipping...")
                    continue

                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
                optimizer.step()

                epoch_loss += loss.item()
                successful_batches += 1
                progress_bar.set_postfix({
                    "Loss": f"{loss.item():.4f}",
                    "Success": f"{successful_batches}/{batch_idx+1}"
                })

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue

        if successful_batches > 0:
            scheduler.step()
            avg_loss = epoch_loss / successful_batches
            print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.2e}")
        else:
            print(f"Epoch {epoch+1} failed - no successful batches")

        if (epoch + 1) % save_every == 0:
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': unet.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': avg_loss if successful_batches > 0 else float('inf'),
            }
            torch.save(checkpoint, f'output/checkpoint_epoch_{epoch+1}.pt')
            print(f"Checkpoint saved at epoch {epoch+1}")

    print("Training completed!")
    return vae, text_encoder, tokenizer, unet, noise_scheduler

if __name__ == "__main__":
    try:
        vae, text_encoder, tokenizer, trained_unet, scheduler = train_diffusion_model()
        print("Training completed successfully!")

        generate_test_images(vae, text_encoder, tokenizer, trained_unet, scheduler)
    
        torch.save(trained_unet.state_dict(), "output/unet_weights.pt")
        print(" UNet weights saved to output/unet_weights.pt")

    except Exception as e:
        print(f" Training failed: {e}")

Using device: cuda
Loaded 37 image-caption pairs
Initialized dataset with 37 valid image-caption pairs
Starting training...


Epoch 1/600: 100%|█████████████████████████████████████████| 37/37 [00:12<00:00,  3.06it/s, Loss=0.8852, Success=37/37]


Epoch 1 completed. Average Loss: 0.9080, LR: 1.00e-04


Epoch 2/600: 100%|█████████████████████████████████████████| 37/37 [00:10<00:00,  3.40it/s, Loss=0.4182, Success=37/37]


Epoch 2 completed. Average Loss: 0.6114, LR: 1.00e-04


...

Epoch 598 completed. Average Loss: 0.0129, LR: 3.55e-05


Epoch 599/600: 100%|███████████████████████████████████████| 37/37 [00:11<00:00,  3.21it/s, Loss=0.0021, Success=37/37]


Epoch 599 completed. Average Loss: 0.0213, LR: 3.54e-05


Epoch 600/600: 100%|███████████████████████████████████████| 37/37 [00:11<00:00,  3.34it/s, Loss=0.0056, Success=37/37]


Epoch 600 completed. Average Loss: 0.0330, LR: 3.52e-05
Checkpoint saved at epoch 600
Training completed!
✅ Training completed successfully!
Using device: cuda
🎨 Generating test images directly...
Generating: a pixel art cat
❌ Error generating 'a pixel art cat': Expected a 'cpu' device type for generator but found 'cuda'
Error type: RuntimeError
Generating: a pixel art house
❌ Error generating 'a pixel art house': Expected a 'cpu' device type for generator but found 'cuda'
Error type: RuntimeError
Generating: a pixel art tree
❌ Error generating 'a pixel art tree': Expected a 'cpu' device type for generator but found 'cuda'
Error type: RuntimeError
Generating: james graham ballard, highrise, sustainability, octane render, highly detailed
❌ Error generating 'james graham ballard, highrise, sustainability, octane render, highly detailed': Expected a 'cpu' device type for generator but found 'cuda'
Error type: RuntimeError
Generating: pixel art landscape
❌ Error generating 'pixel art lands

In [3]:
import torch
import random
import numpy as np

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
# Sinusoidal timestep embedding for diffusion steps
def get_timestep_embedding(timesteps, embedding_dim):
    half_dim = embedding_dim // 2
    emb = torch.exp(
        torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) *
        -(torch.log(torch.tensor(10000.0)) / half_dim)
    )
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # Handle odd embedding dimensions
        emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
    return emb

# Residual block with time and context embeddings
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_emb_dim, context_dim=None):
        super().__init__()
        self.norm1 = nn.GroupNorm(min(32, in_channels), in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.norm2 = nn.GroupNorm(min(32, out_channels), out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_mlp = nn.Linear(time_emb_dim, out_channels)
        self.context_proj = nn.Linear(context_dim, out_channels) if context_dim else None
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, t_emb, context=None):
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)

        # Add time embedding
        t_proj = self.time_mlp(t_emb)[:, :, None, None]
        h = h + t_proj

        # Add context embedding if available
        if self.context_proj is not None and context is not None:
            context_pooled = context.mean(dim=1)  # [batch, context_dim]
            context_proj = self.context_proj(context_pooled)[:, :, None, None]
            h = h + context_proj

        h = self.norm2(h)
        h = F.silu(h)
        h = self.conv2(h)

        return h + self.shortcut(x)

# Cross-attention to integrate text embeddings
class CrossAttention(nn.Module):
    def __init__(self, channels, context_dim):
        super().__init__()
        self.channels = channels
        self.query = nn.Linear(channels, channels)
        self.key = nn.Linear(context_dim, channels)
        self.value = nn.Linear(context_dim, channels)
        self.out = nn.Linear(channels, channels)
        self.norm = nn.LayerNorm(channels)

    def forward(self, x, context):
        if context is None:
            return x

        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm(x_flat)

        q = self.query(x_norm)  # [B, H*W, C]
        k = self.key(context)   # [B, seq_len, C]
        v = self.value(context) # [B, seq_len, C]

        scale = (C ** -0.5)
        attn_weights = torch.bmm(q, k.transpose(1, 2)) * scale
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_out = torch.bmm(attn_weights, v)
        attn_out = self.out(attn_out)

        attn_out = attn_out.reshape(B, H, W, C).permute(0, 3, 1, 2)
        return x + attn_out

# Self-attention block for image features
class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.qkv = nn.Conv2d(channels, channels * 3, 1)
        self.proj = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        qkv = self.qkv(h).reshape(B, 3, C, H * W)
        q, k, v = qkv[:, 0], qkv[:, 1], qkv[:, 2]

        scale = (C ** -0.5)
        attn = torch.bmm(q.transpose(1, 2), k) * scale
        attn = F.softmax(attn, dim=-1)

        out = torch.bmm(v, attn.transpose(1, 2))
        out = out.reshape(B, C, H, W)
        return self.proj(out) + x

# U-Net model updated for 256x256 latents
class UNetConditional(nn.Module):
    def __init__(self, in_channels=4, base_channels=128, context_dim=768):
        super().__init__()
        self.time_emb_dim = base_channels * 4
        from types import SimpleNamespace
        self.config = SimpleNamespace()
        self.config._diffusers_version = "0.34.0"
        self.config.in_channels = in_channels
        self.config.out_channels = in_channels
        self.config.sample_size = 256  # Updated for 256x256 latents
        self.config.layers_per_block = 2
        self.config.block_out_channels = [base_channels, base_channels * 2, base_channels * 4, base_channels * 8]
        self.config.attention_head_dim = 8
        self.config.cross_attention_dim = context_dim

        # Time embedding MLP
        self.time_mlp = nn.Sequential(
            nn.Linear(base_channels, self.time_emb_dim),
            nn.SiLU(),
            nn.Linear(self.time_emb_dim, self.time_emb_dim),
        )

        # Input projection
        self.input_conv = nn.Conv2d(in_channels, base_channels, 3, padding=1)

        # Encoder
        self.down1 = ResidualBlock(base_channels, base_channels * 2, self.time_emb_dim, context_dim)
        self.downsample1 = nn.Conv2d(base_channels * 2, base_channels * 2, 3, stride=2, padding=1)
        self.cross1 = CrossAttention(base_channels * 2, context_dim)

        self.down2 = ResidualBlock(base_channels * 2, base_channels * 4, self.time_emb_dim, context_dim)
        self.downsample2 = nn.Conv2d(base_channels * 4, base_channels * 4, 3, stride=2, padding=1)
        self.cross2 = CrossAttention(base_channels * 4, context_dim)

        self.down3 = ResidualBlock(base_channels * 4, base_channels * 8, self.time_emb_dim, context_dim)
        self.downsample3 = nn.Conv2d(base_channels * 8, base_channels * 8, 3, stride=2, padding=1)
        self.cross3 = CrossAttention(base_channels * 8, context_dim)

        # Middle
        self.middle1 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)
        self.middle_attn = AttentionBlock(base_channels * 8)
        self.middle2 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)

        # Decoder
        self.up3 = ResidualBlock(base_channels * 16, base_channels * 4, self.time_emb_dim, context_dim)
        self.upsample3 = nn.ConvTranspose2d(base_channels * 4, base_channels * 4, 4, stride=2, padding=1)
        self.cross_up3 = CrossAttention(base_channels * 4, context_dim)

        self.up2 = ResidualBlock(base_channels * 8, base_channels * 2, self.time_emb_dim, context_dim)
        self.upsample2 = nn.ConvTranspose2d(base_channels * 2, base_channels * 2, 4, stride=2, padding=1)
        self.cross_up2 = CrossAttention(base_channels * 2, context_dim)

        self.up1 = ResidualBlock(base_channels * 4, base_channels, self.time_emb_dim, context_dim)
        self.upsample1 = nn.ConvTranspose2d(base_channels, base_channels, 4, stride=2, padding=1)

        # Output
        self.output_conv = nn.Sequential(
            nn.GroupNorm(min(32, base_channels), base_channels),
            nn.SiLU(),
            nn.Conv2d(base_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t, context, cfg_scale=1.0):
        t_emb = get_timestep_embedding(t, self.time_emb_dim // 4)
        t_emb = self.time_mlp(t_emb)

        def denoise(x, t_emb, context):
            h = self.input_conv(x)

            # Encoder
            h1 = self.down1(h, t_emb, context)
            h1_cross = self.cross1(h1, context)
            h1_down = self.downsample1(h1_cross)

            h2 = self.down2(h1_down, t_emb, context)
            h2_cross = self.cross2(h2, context)
            h2_down = self.downsample2(h2_cross)

            h3 = self.down3(h2_down, t_emb, context)
            h3_cross = self.cross3(h3, context)
            h3_down = self.downsample3(h3_cross)

            # Middle
            h_mid = self.middle1(h3_down, t_emb, context)
            h_mid = self.middle_attn(h_mid)
            h_mid = self.middle2(h_mid, t_emb, context)

            # Decoder
            h3_cross_resized = F.interpolate(h3_cross, size=h_mid.shape[-2:], mode='nearest')
            h = self.up3(torch.cat([h_mid, h3_cross_resized], dim=1), t_emb, context)
            h = self.upsample3(h)
            h = self.cross_up3(h, context)

            h2_cross_resized = F.interpolate(h2_cross, size=h.shape[-2:], mode='nearest')
            h = self.up2(torch.cat([h, h2_cross_resized], dim=1), t_emb, context)
            h = self.upsample2(h)
            h = self.cross_up2(h, context)

            h1_cross_resized = F.interpolate(h1_cross, size=h.shape[-2:], mode='nearest')
            h = self.up1(torch.cat([h, h1_cross_resized], dim=1), t_emb, context)
            h = self.upsample1(h)

            return self.output_conv(h)

        if cfg_scale == 1.0 or context is None:
            return denoise(x, t_emb, context)

        uncond = denoise(x, t_emb, context=None)
        cond = denoise(x, t_emb, context)
        return uncond + cfg_scale * (cond - uncond)


In [4]:
import torch
import torch.nn.functional as F
from diffusers import AutoencoderKL, DDPMScheduler
from transformers import CLIPTokenizer, CLIPTextModel
from tqdm import tqdm
import numpy as np
from PIL import Image
# Import UNetConditional from the cell where it's defined


def generate_images_direct(unet_path="output/checkpoint_epoch_600.pt", device="cuda"):
    """Generate 256x256 images directly without using StableDiffusionPipeline"""
    seed_everything(42)
    print(f"Using device: {device}")

    # Load components
    print("Loading VAE...")
    vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
    vae = vae.eval().requires_grad_(False).to(device)

    print("Loading tokenizer and text encoder...")
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = text_encoder.eval().requires_grad_(False).to(device)

    # Load your trained UNet with proper device handling
    print("Loading trained UNet...")
    # Correctly initialize the UNet with base_channels=128, using the class from the other cell
    unet = UNetConditional(in_channels=4, base_channels=128, context_dim=768)
    checkpoint = torch.load(unet_path, map_location=device, weights_only=True)
    # Load only the model_state_dict from the checkpoint
    unet.load_state_dict(checkpoint['model_state_dict'])
    unet = unet.eval().to(device)

    # Create scheduler
    scheduler = DDPMScheduler(num_train_timesteps=1000)

    # Test prompts
    test_prompts = ["A futuristic smartphone"]

    print("🎨 Generating 256x256 images directly...")

    for i, prompt in enumerate(test_prompts):
        print(f"Generating: {prompt}")

        try:
            with torch.no_grad():
                # Encode prompt - ensure all tensors on correct device
                inputs = tokenizer(
                    prompt,
                    padding="max_length",
                    truncation=True,
                    max_length=77,
                    return_tensors="pt"
                )
                # Move tokenizer outputs to device
                inputs = {k: v.to(device) for k, v in inputs.items()}

                text_embeddings = text_encoder(**inputs).last_hidden_state
                print(f"Text embeddings shape: {text_embeddings.shape}, device: {text_embeddings.device}")

                # Create random latents for 256x256 output (256/8 = 32 due to VAE scaling)
                latents = torch.randn(1, 4, 32, 32, device=device, dtype=torch.float32)
                print(f"Initial latents shape: {latents.shape}, device: {latents.device}")

                # Set timesteps
                scheduler.set_timesteps(1000)

                # Denoising loop
                for t in tqdm(scheduler.timesteps, desc=f"Denoising {prompt}"):
                    # Ensure timestep is on correct device
                    t_tensor = torch.tensor([t], device=device, dtype=torch.long)

                    # Predict noise - all inputs should be on same device
                    # Pass text_embeddings as the context argument
                    noise_pred = unet(latents, t_tensor, context=text_embeddings)

                    # Remove noise
                    latents = scheduler.step(noise_pred, t, latents).prev_sample

                print(f"Final latents shape: {latents.shape}")

                # Decode latents to image
                latents = latents / 0.18215

                # No need to resize since UNet was trained on 256x256 (32x32 latents)
                images = vae.decode(latents).sample

                # Convert to PIL
                images = (images + 1) / 2  # Denormalize
                images = images.clamp(0, 1)
                images = images.cpu().permute(0, 2, 3, 1).numpy()

                image = Image.fromarray((images[0] * 255).astype(np.uint8))

                # Save
                filename = f"output/generated_256_{i+1}_{prompt.replace(' ', '_')}.png"
                image.save(filename)
                print(f"✅ Saved: {filename}")

        except Exception as e:
            print(f"❌ Error generating '{prompt}': {e}")
            print(f"Error type: {type(e).__name__}")
            # Continue with next prompt
            continue

In [11]:
generate_images_direct()


Using device: cuda
Loading VAE...
Loading tokenizer and text encoder...
Loading trained UNet...


FileNotFoundError: [Errno 2] No such file or directory: 'output/checkpoint_epoch_600.pt'

In [ ]:
from diffusers import StableDiffusionPipeline, AutoencoderKL, DDIMScheduler
from transformers import CLIPTokenizer, CLIPTextModel
import torch
import torch
import random
import numpy as np

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
# Sinusoidal timestep embedding for diffusion steps
def get_timestep_embedding(timesteps, embedding_dim):
    half_dim = embedding_dim // 2
    emb = torch.exp(
        torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) *
        -(torch.log(torch.tensor(10000.0)) / half_dim)
    )
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # Handle odd embedding dimensions
        emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
    return emb

# Residual block with time and context embeddings
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_emb_dim, context_dim=None):
        super().__init__()
        self.norm1 = nn.GroupNorm(min(32, in_channels), in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.norm2 = nn.GroupNorm(min(32, out_channels), out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_mlp = nn.Linear(time_emb_dim, out_channels)
        self.context_proj = nn.Linear(context_dim, out_channels) if context_dim else None
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, t_emb, context=None):
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)

        # Add time embedding
        t_proj = self.time_mlp(t_emb)[:, :, None, None]
        h = h + t_proj

        # Add context embedding if available
        if self.context_proj is not None and context is not None:
            context_pooled = context.mean(dim=1)  # [batch, context_dim]
            context_proj = self.context_proj(context_pooled)[:, :, None, None]
            h = h + context_proj

        h = self.norm2(h)
        h = F.silu(h)
        h = self.conv2(h)

        return h + self.shortcut(x)

# Cross-attention to integrate text embeddings
class CrossAttention(nn.Module):
    def __init__(self, channels, context_dim):
        super().__init__()
        self.channels = channels
        self.query = nn.Linear(channels, channels)
        self.key = nn.Linear(context_dim, channels)
        self.value = nn.Linear(context_dim, channels)
        self.out = nn.Linear(channels, channels)
        self.norm = nn.LayerNorm(channels)

    def forward(self, x, context):
        if context is None:
            return x

        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm(x_flat)

        q = self.query(x_norm)  # [B, H*W, C]
        k = self.key(context)   # [B, seq_len, C]
        v = self.value(context) # [B, seq_len, C]

        scale = (C ** -0.5)
        attn_weights = torch.bmm(q, k.transpose(1, 2)) * scale
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_out = torch.bmm(attn_weights, v)
        attn_out = self.out(attn_out)

        attn_out = attn_out.reshape(B, H, W, C).permute(0, 3, 1, 2)
        return x + attn_out

# Self-attention block for image features
class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.qkv = nn.Conv2d(channels, channels * 3, 1)
        self.proj = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        qkv = self.qkv(h).reshape(B, 3, C, H * W)
        q, k, v = qkv[:, 0], qkv[:, 1], qkv[:, 2]

        scale = (C ** -0.5)
        attn = torch.bmm(q.transpose(1, 2), k) * scale
        attn = F.softmax(attn, dim=-1)

        out = torch.bmm(v, attn.transpose(1, 2))
        out = out.reshape(B, C, H, W)
        return self.proj(out) + x

# U-Net model updated for 256x256 latents
class UNetConditional(nn.Module):
    def __init__(self, in_channels=4, base_channels=128, context_dim=768):
        super().__init__()
        self.time_emb_dim = base_channels * 4
        from types import SimpleNamespace
        self.config = SimpleNamespace()
        self.config._diffusers_version = "0.34.0"
        self.config.in_channels = in_channels
        self.config.out_channels = in_channels
        self.config.sample_size = 256  # Updated for 256x256 latents
        self.config.layers_per_block = 2
        self.config.block_out_channels = [base_channels, base_channels * 2, base_channels * 4, base_channels * 8]
        self.config.attention_head_dim = 8
        self.config.cross_attention_dim = context_dim

        # Time embedding MLP
        self.time_mlp = nn.Sequential(
            nn.Linear(base_channels, self.time_emb_dim),
            nn.SiLU(),
            nn.Linear(self.time_emb_dim, self.time_emb_dim),
        )

        # Input projection
        self.input_conv = nn.Conv2d(in_channels, base_channels, 3, padding=1)

        # Encoder
        self.down1 = ResidualBlock(base_channels, base_channels * 2, self.time_emb_dim, context_dim)
        self.downsample1 = nn.Conv2d(base_channels * 2, base_channels * 2, 3, stride=2, padding=1)
        self.cross1 = CrossAttention(base_channels * 2, context_dim)

        self.down2 = ResidualBlock(base_channels * 2, base_channels * 4, self.time_emb_dim, context_dim)
        self.downsample2 = nn.Conv2d(base_channels * 4, base_channels * 4, 3, stride=2, padding=1)
        self.cross2 = CrossAttention(base_channels * 4, context_dim)

        self.down3 = ResidualBlock(base_channels * 4, base_channels * 8, self.time_emb_dim, context_dim)
        self.downsample3 = nn.Conv2d(base_channels * 8, base_channels * 8, 3, stride=2, padding=1)
        self.cross3 = CrossAttention(base_channels * 8, context_dim)

        # Middle
        self.middle1 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)
        self.middle_attn = AttentionBlock(base_channels * 8)
        self.middle2 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)

        # Decoder
        self.up3 = ResidualBlock(base_channels * 16, base_channels * 4, self.time_emb_dim, context_dim)
        self.upsample3 = nn.ConvTranspose2d(base_channels * 4, base_channels * 4, 4, stride=2, padding=1)
        self.cross_up3 = CrossAttention(base_channels * 4, context_dim)

        self.up2 = ResidualBlock(base_channels * 8, base_channels * 2, self.time_emb_dim, context_dim)
        self.upsample2 = nn.ConvTranspose2d(base_channels * 2, base_channels * 2, 4, stride=2, padding=1)
        self.cross_up2 = CrossAttention(base_channels * 2, context_dim)

        self.up1 = ResidualBlock(base_channels * 4, base_channels, self.time_emb_dim, context_dim)
        self.upsample1 = nn.ConvTranspose2d(base_channels, base_channels, 4, stride=2, padding=1)

        # Output
        self.output_conv = nn.Sequential(
            nn.GroupNorm(min(32, base_channels), base_channels),
            nn.SiLU(),
            nn.Conv2d(base_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t, context, cfg_scale=1.0):
        t_emb = get_timestep_embedding(t, self.time_emb_dim // 4)
        t_emb = self.time_mlp(t_emb)

        def denoise(x, t_emb, context):
            h = self.input_conv(x)

            # Encoder
            h1 = self.down1(h, t_emb, context)
            h1_cross = self.cross1(h1, context)
            h1_down = self.downsample1(h1_cross)

            h2 = self.down2(h1_down, t_emb, context)
            h2_cross = self.cross2(h2, context)
            h2_down = self.downsample2(h2_cross)

            h3 = self.down3(h2_down, t_emb, context)
            h3_cross = self.cross3(h3, context)
            h3_down = self.downsample3(h3_cross)

            # Middle
            h_mid = self.middle1(h3_down, t_emb, context)
            h_mid = self.middle_attn(h_mid)
            h_mid = self.middle2(h_mid, t_emb, context)

            # Decoder
            h3_cross_resized = F.interpolate(h3_cross, size=h_mid.shape[-2:], mode='nearest')
            h = self.up3(torch.cat([h_mid, h3_cross_resized], dim=1), t_emb, context)
            h = self.upsample3(h)
            h = self.cross_up3(h, context)

            h2_cross_resized = F.interpolate(h2_cross, size=h.shape[-2:], mode='nearest')
            h = self.up2(torch.cat([h, h2_cross_resized], dim=1), t_emb, context)
            h = self.upsample2(h)
            h = self.cross_up2(h, context)

            h1_cross_resized = F.interpolate(h1_cross, size=h.shape[-2:], mode='nearest')
            h = self.up1(torch.cat([h, h1_cross_resized], dim=1), t_emb, context)
            h = self.upsample1(h)

            return self.output_conv(h)

        if cfg_scale == 1.0 or context is None:
            return denoise(x, t_emb, context)

        uncond = denoise(x, t_emb, context=None)
        cond = denoise(x, t_emb, context)
        return uncond + cfg_scale * (cond - uncond)
# Your UNetConditional and related classes (ResidualBlock, CrossAttention, AttentionBlock, etc.) go here
# ... (Your provided UNet code)

import torch
import torch.nn as nn
from diffusers import StableDiffusionPipeline, AutoencoderKL, DDIMScheduler, UNet2DConditionModel
from transformers import CLIPTokenizer, CLIPTextModel
from tqdm import tqdm
import numpy as np
from PIL import Image
import random
import os



import torch
from diffusers import StableDiffusionPipeline, AutoencoderKL, DDIMScheduler, UNet2DConditionModel
from transformers import CLIPTokenizer, CLIPTextModel
from tqdm import tqdm
import numpy as np
from PIL import Image
import random
import os



# Set random seed
def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def create_and_save_pipeline(unet_path="output/checkpoint_epoch_600.pt", output_dir="KahabMiniGenT2Im-v1", device="cuda"):
    seed_everything(42)

    # Load components
    print("Loading VAE...")
    vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16)
    vae = vae.eval().requires_grad_(False).to(device)

    print("Loading tokenizer and text encoder...")
    tokenizer = CLIPTokenizer.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="text_encoder", torch_dtype=torch.float16)
    text_encoder = text_encoder.eval().requires_grad_(False).to(device)

    print("Loading scheduler...")
    scheduler = DDIMScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

    print("Loading trained UNet...")
    unet = UNetConditional(in_channels=4, base_channels=128, context_dim=768)
    checkpoint = torch.load(unet_path, map_location=device, weights_only=True)
    try:
        unet.load_state_dict(checkpoint['model_state_dict'])
    except Exception as e:
        print(f"❌ Error loading UNet weights: {e}")
        print("Checkpoint keys (first 10):", list(checkpoint['model_state_dict'].keys())[:10])
        print("UNet keys (first 10):", list(unet.state_dict().keys())[:10])
        return
    unet = unet.eval().to(device, dtype=torch.float32)

    # Create diffusers UNet
    print("Creating UNet2DConditionModel for pipeline...")
    unet_config = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet").config
    unet_diffusers = UNet2DConditionModel(**unet_config).to(device, dtype=torch.float32)
    try:
        unet_diffusers.load_state_dict(unet.state_dict(), strict=False)
    except Exception as e:
        print(f"❌ Error transferring UNet weights: {e}")
        print("Custom UNet keys (first 10):", list(unet.state_dict().keys())[:10])
        print("Diffusers UNet keys (first 10):", list(unet_diffusers.state_dict().keys())[:10])
        return

    # Minimal pipeline
    class CustomStableDiffusionPipeline(StableDiffusionPipeline):
        def __call__(self, prompt, num_inference_steps=50, guidance_scale=7.5, **kwargs):
            device = self.unet.device
            text_inputs = self.tokenizer(
                prompt, padding="max_length", max_length=self.tokenizer.model_max_length, truncation=True, return_tensors="pt"
            )
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            with torch.no_grad():
                text_embeddings = self.text_encoder(text_inputs["input_ids"])[0].to(torch.float32)

            empty_inputs = self.tokenizer(
                "", padding="max_length", max_length=self.tokenizer.model_max_length, truncation=True, return_tensors="pt"
            )
            empty_inputs = {k: v.to(device) for k, v in empty_inputs.items()}
            with torch.no_grad():
                empty_embeddings = self.text_encoder(empty_inputs["input_ids"])[0].to(torch.float32)

            latents = torch.randn(
                (1, self.unet.config.in_channels, 32, 32), device=device, dtype=torch.float32
            )
            latents = latents * self.scheduler.init_noise_sigma

            self.scheduler.set_timesteps(num_inference_steps)

            for t in tqdm(self.scheduler.timesteps, desc=f"Denoising {prompt}"):
                latent_model_input = self.scheduler.scale_model_input(latents, t)
                t_tensor = torch.tensor([t], device=device, dtype=torch.float32)
                with torch.no_grad():
                    noise_pred = self.unet(latent_model_input, t_tensor, encoder_hidden_states=text_embeddings)['sample']
                latents = self.scheduler.step(noise_pred, t, latents).prev_sample

            latents = latents / 0.18215
            latents = latents.to(dtype=torch.float16)
            with torch.no_grad():
                images = self.vae.decode(latents).sample
            images = (images / 2 + 0.5).clamp(0, 1)
            images = images.cpu().permute(0, 2, 3, 1).numpy()
            images = [Image.fromarray((img * 255).astype(np.uint8)) for img in images]
            return {"images": images}

    # Create pipeline
    try:
        pipeline = CustomStableDiffusionPipeline(
            vae=vae,
            text_encoder=text_encoder,
            tokenizer=tokenizer,
            unet=unet_diffusers,
            scheduler=scheduler,
            safety_checker=None,
            feature_extractor=None
        ).to(device)
        print("✅ Pipeline created successfully")
        print("Pipeline components:", list(pipeline.components.keys()))
    except Exception as e:
        print(f"❌ Error creating pipeline: {e}")
        return

    # Test pipeline
    try:
        print("Testing pipeline...")
        image = pipeline("A futuristic smartphone", num_inference_steps=50, guidance_scale=7.5)["images"][0]
        image.save("test_pipeline_output.png")
        print("✅ Test image generated successfully")
    except Exception as e:
        print(f"❌ Error testing pipeline: {e}")
        return

    # Save pipeline
    try:
        os.makedirs(output_dir, exist_ok=True)
        pipeline.save_pretrained(output_dir, safe_serialization=True)
        print(f"✅ Pipeline saved to {output_dir}")
    except Exception as e:
        print(f"❌ Error saving pipeline: {e}")
        return

if __name__ == "__main__":
    create_and_save_pipeline()

Loading VAE...
Loading tokenizer and text encoder...
Loading scheduler...
Loading trained UNet...
Creating UNet2DConditionModel for pipeline...


You have disabled the safety checker for <class '__main__.create_and_save_pipeline.<locals>.CustomStableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


✅ Pipeline created successfully
Pipeline components: ['vae', 'text_encoder', 'tokenizer', 'unet', 'scheduler', 'safety_checker', 'feature_extractor', 'image_encoder']
Testing pipeline...


Denoising A futuristic smartphone:  46%|█████████████████████▌                         | 23/50 [00:19<00:24,  1.12it/s]

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, DDPMScheduler
import math
import glob
import random
import json

# -----------------------
# Advanced U-Net
# -----------------------
class SinusoidalTimeEmbedding(nn.Module):
    def _init_(self, dim):
        super()._init_()
        self.linear1 = nn.Linear(dim, dim * 4)
        self.linear2 = nn.Linear(dim * 4, dim)

    def forward(self, t):
        half_dim = self.linear1.in_features // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=t.device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return self.linear2(F.silu(self.linear1(emb)))

class CrossAttentionBlock(nn.Module):
    def _init_(self, dim, context_dim):
        super()._init_()
        self.norm = nn.LayerNorm(dim)
        self.q = nn.Linear(dim, dim)
        self.k = nn.Linear(context_dim, dim)
        self.v = nn.Linear(context_dim, dim)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x, context):
        b, n, c = x.shape
        q = self.q(self.norm(x))
        k = self.k(context)
        v = self.v(context)
        attn = torch.softmax(q @ k.transpose(-2, -1) / math.sqrt(c), dim=-1)
        return self.proj(attn @ v)

class ResBlock(nn.Module):
    def _init_(self, in_ch, out_ch, time_emb_dim):
        super()._init_()
        self.norm1 = nn.GroupNorm(32, in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.time_emb_proj = nn.Linear(time_emb_dim, out_ch)
        self.norm2 = nn.GroupNorm(32, out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.skip = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x, t_emb):
        h = F.silu(self.norm1(x))
        h = self.conv1(h)
        h += self.time_emb_proj(t_emb).unsqueeze(-1).unsqueeze(-1)
        h = F.silu(self.norm2(h))
        h = self.conv2(h)
        return h + self.skip(x)

class AttentionBlock(nn.Module):
    def _init_(self, dim):
        super()._init_()
        self.norm = nn.GroupNorm(32, dim)
        self.q = nn.Conv2d(dim, dim, 1)
        self.k = nn.Conv2d(dim, dim, 1)
        self.v = nn.Conv2d(dim, dim, 1)
        self.proj = nn.Conv2d(dim, dim, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        q = self.q(h).reshape(B, C, H * W).permute(0, 2, 1)
        k = self.k(h).reshape(B, C, H * W)
        v = self.v(h).reshape(B, C, H * W).permute(0, 2, 1)
        attn = torch.softmax(q @ k / math.sqrt(C), dim=-1)
        out = attn @ v
        out = out.permute(0, 2, 1).reshape(B, C, H, W)
        return x + self.proj(out)

class AdvancedUNet(nn.Module):
    def _init_(self, in_ch=4, base_ch=320, context_dim=768, time_dim=1280):
        super()._init_()
        self.time_mlp = SinusoidalTimeEmbedding(time_dim)
        self.conv_in = nn.Conv2d(in_ch, base_ch, 3, padding=1)
        self.down1 = nn.Sequential(ResBlock(base_ch, base_ch, time_dim), AttentionBlock(base_ch), nn.AvgPool2d(2))
        self.down2 = nn.Sequential(ResBlock(base_ch, base_ch * 2, time_dim), AttentionBlock(base_ch * 2), nn.AvgPool2d(2))
        self.mid = nn.Sequential(ResBlock(base_ch * 2, base_ch * 4, time_dim), AttentionBlock(base_ch * 4), ResBlock(base_ch * 4, base_ch * 2, time_dim))
        self.cross_attn = CrossAttentionBlock(base_ch * 2, context_dim)
        self.up1 = nn.Sequential(ResBlock(base_ch * 2, base_ch, time_dim), AttentionBlock(base_ch), nn.Upsample(scale_factor=2, mode="nearest"))
        self.up2 = nn.Sequential(ResBlock(base_ch, base_ch, time_dim), AttentionBlock(base_ch), nn.Upsample(scale_factor=2, mode="nearest"))
        self.conv_out = nn.Sequential(nn.GroupNorm(32, base_ch), nn.SiLU(), nn.Conv2d(base_ch, in_ch, 3, padding=1))

    def forward(self, x, t, context):
        t_emb = self.time_mlp(t)
        x = self.conv_in(x)
        d1 = self.down1(x)
        d2 = self.down2(d1)
        mid = self.mid(d2)
        B, C, H, W = mid.shape
        context_proj = self.cross_attn(mid.view(B, C, -1).permute(0, 2, 1), context)
        mid = mid + context_proj.permute(0, 2, 1).view(B, C, H, W)
        u1 = self.up1(mid)
        u2 = self.up2(u1)
        return self.conv_out(u2)

# -----------------------
# Dataset & Preprocessing
# -----------------------
class FaceLatentDataset(Dataset):
    def _init_(self, image_dir, caption_file, vae, tokenizer, text_encoder, size=512):
        self.image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))[:10]
        self.caption_data = json.load(open(caption_file))
        self.vae = vae
        self.tokenizer = tokenizer
        self.text_encoder = text_encoder
        self.transform = transforms.Compose([
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

    def _len_(self):
        return len(self.image_paths)

    def _getitem_(self, idx):
        path = self.image_paths[idx]
        image = self.transform(Image.open(path).convert("RGB")).unsqueeze(0).cuda()
        with torch.no_grad():
            latent = self.vae.encode(image).latent_dist.sample() * 0.18215
            text = self.caption_data[os.path.basename(path)]
            text_token = self.tokenizer([text], padding="max_length", max_length=77, return_tensors="pt").input_ids.cuda()
            text_embedding = self.text_encoder(text_token)[0]
        return latent.squeeze(0), text_embedding.squeeze(0)

# -----------------------
# Training Setup
# -----------------------
def train(model, dataloader, noise_scheduler, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        for latents, text_embeds in dataloader:
            noise = torch.randn_like(latents)
            t = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=latents.device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, t)
            preds = model(noisy_latents, t, text_embeds)
            loss = F.mse_loss(preds, noise)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(f"[Epoch {epoch+1}] Loss: {loss.item():.4f}")

# -----------------------
# MAIN
# -----------------------
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-5", subfolder="vae").to(device)
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
    model = AdvancedUNet().to(device)
    dataset = FaceLatentDataset("/content/images/", "/content/exprtion.json", vae, tokenizer, text_encoder)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    train(model, dataloader, noise_scheduler, optimizer, epochs=10)
    torch.save(model.state_dict(), "my_custom_unet.pth")


main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: CompVis/stable-diffusion-v1-5 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `token` or log in with `huggingface-cli login`.

In [1]:
import os
import shutil

# Define source and destination paths
image_source_dir = '.'  # Current directory where image files are located
image_dest_dir = 'images' # Destination folder for image files
json_source_path = 'captions.json' # Source path for the JSON file
json_dest_path = '../captions.json' # Destination path for the JSON file (one level up)

# Create the destination directory for images if it doesn't exist
os.makedirs(image_dest_dir, exist_ok=True)

# Move image files
for filename in os.listdir(image_source_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')): # Add other image extensions if needed
        shutil.move(os.path.join(image_source_dir, filename), os.path.join(image_dest_dir, filename))
        print(f"Moved {filename} to {image_dest_dir}")

# Move the JSON file
if os.path.exists(json_source_path):
    shutil.move(json_source_path, json_dest_path)
    print(f"Moved {json_source_path} to {json_dest_path}")
else:
    print(f"{json_source_path} not found.")

Moved expression_02.jpg to images
Moved expression_07.jpg to images
Moved expression_06.jpg to images
Moved expression_08.jpg to images
Moved expression_01.jpg to images
Moved expression_09.jpg to images
Moved expression_05.jpg to images
Moved expression_04.jpg to images
Moved expression_10.jpg to images
Moved expression_03.jpg to images
captions.json not found.


In [4]:
seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load components
print("Loading VAE...")
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(device).eval().requires_grad_(False)

print("Loading tokenizer and text encoder...")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device).eval().requires_grad_(False)


Loading VAE...
Loading tokenizer and text encoder...


In [5]:
import torch
import random
import numpy as np

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
# Sinusoidal timestep embedding for diffusion steps
def get_timestep_embedding(timesteps, embedding_dim):
    half_dim = embedding_dim // 2
    emb = torch.exp(
        torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) *
        -(torch.log(torch.tensor(10000.0)) / half_dim)
    )
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # Handle odd embedding dimensions
        emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
    return emb

# Residual block with time and context embeddings
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_emb_dim, context_dim=None):
        super().__init__()
        self.norm1 = nn.GroupNorm(min(32, in_channels), in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.norm2 = nn.GroupNorm(min(32, out_channels), out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_mlp = nn.Linear(time_emb_dim, out_channels)
        self.context_proj = nn.Linear(context_dim, out_channels) if context_dim else None
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, t_emb, context=None):
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)

        # Add time embedding
        t_proj = self.time_mlp(t_emb)[:, :, None, None]
        h = h + t_proj

        # Add context embedding if available
        if self.context_proj is not None and context is not None:
            context_pooled = context.mean(dim=1)  # [batch, context_dim]
            context_proj = self.context_proj(context_pooled)[:, :, None, None]
            h = h + context_proj

        h = self.norm2(h)
        h = F.silu(h)
        h = self.conv2(h)

        return h + self.shortcut(x)

# Cross-attention to integrate text embeddings
class CrossAttention(nn.Module):
    def __init__(self, channels, context_dim):
        super().__init__()
        self.channels = channels
        self.query = nn.Linear(channels, channels)
        self.key = nn.Linear(context_dim, channels)
        self.value = nn.Linear(context_dim, channels)
        self.out = nn.Linear(channels, channels)
        self.norm = nn.LayerNorm(channels)

    def forward(self, x, context):
        if context is None:
            return x

        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm(x_flat)

        q = self.query(x_norm)  # [B, H*W, C]
        k = self.key(context)   # [B, seq_len, C]
        v = self.value(context) # [B, seq_len, C]

        scale = (C ** -0.5)
        attn_weights = torch.bmm(q, k.transpose(1, 2)) * scale
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_out = torch.bmm(attn_weights, v)
        attn_out = self.out(attn_out)

        attn_out = attn_out.reshape(B, H, W, C).permute(0, 3, 1, 2)
        return x + attn_out

# Self-attention block for image features
class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.qkv = nn.Conv2d(channels, channels * 3, 1)
        self.proj = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        qkv = self.qkv(h).reshape(B, 3, C, H * W)
        q, k, v = qkv[:, 0], qkv[:, 1], qkv[:, 2]

        scale = (C ** -0.5)
        attn = torch.bmm(q.transpose(1, 2), k) * scale
        attn = F.softmax(attn, dim=-1)

        out = torch.bmm(v, attn.transpose(1, 2))
        out = out.reshape(B, C, H, W)
        return self.proj(out) + x

# U-Net model updated for 256x256 latents
class UNetConditional(nn.Module):
    def __init__(self, in_channels=4, base_channels=128, context_dim=768):
        super().__init__()
        self.time_emb_dim = base_channels * 4
        from types import SimpleNamespace
        self.config = SimpleNamespace()
        self.config._diffusers_version = "0.34.0"
        self.config.in_channels = in_channels
        self.config.out_channels = in_channels
        self.config.sample_size = 256  # Updated for 256x256 latents
        self.config.layers_per_block = 2
        self.config.block_out_channels = [base_channels, base_channels * 2, base_channels * 4, base_channels * 8]
        self.config.attention_head_dim = 8
        self.config.cross_attention_dim = context_dim

        # Time embedding MLP
        self.time_mlp = nn.Sequential(
            nn.Linear(base_channels, self.time_emb_dim),
            nn.SiLU(),
            nn.Linear(self.time_emb_dim, self.time_emb_dim),
        )

        # Input projection
        self.input_conv = nn.Conv2d(in_channels, base_channels, 3, padding=1)

        # Encoder
        self.down1 = ResidualBlock(base_channels, base_channels * 2, self.time_emb_dim, context_dim)
        self.downsample1 = nn.Conv2d(base_channels * 2, base_channels * 2, 3, stride=2, padding=1)
        self.cross1 = CrossAttention(base_channels * 2, context_dim)

        self.down2 = ResidualBlock(base_channels * 2, base_channels * 4, self.time_emb_dim, context_dim)
        self.downsample2 = nn.Conv2d(base_channels * 4, base_channels * 4, 3, stride=2, padding=1)
        self.cross2 = CrossAttention(base_channels * 4, context_dim)

        self.down3 = ResidualBlock(base_channels * 4, base_channels * 8, self.time_emb_dim, context_dim)
        self.downsample3 = nn.Conv2d(base_channels * 8, base_channels * 8, 3, stride=2, padding=1)
        self.cross3 = CrossAttention(base_channels * 8, context_dim)

        # Middle
        self.middle1 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)
        self.middle_attn = AttentionBlock(base_channels * 8)
        self.middle2 = ResidualBlock(base_channels * 8, base_channels * 8, self.time_emb_dim, context_dim)

        # Decoder
        self.up3 = ResidualBlock(base_channels * 16, base_channels * 4, self.time_emb_dim, context_dim)
        self.upsample3 = nn.ConvTranspose2d(base_channels * 4, base_channels * 4, 4, stride=2, padding=1)
        self.cross_up3 = CrossAttention(base_channels * 4, context_dim)

        self.up2 = ResidualBlock(base_channels * 8, base_channels * 2, self.time_emb_dim, context_dim)
        self.upsample2 = nn.ConvTranspose2d(base_channels * 2, base_channels * 2, 4, stride=2, padding=1)
        self.cross_up2 = CrossAttention(base_channels * 2, context_dim)

        self.up1 = ResidualBlock(base_channels * 4, base_channels, self.time_emb_dim, context_dim)
        self.upsample1 = nn.ConvTranspose2d(base_channels, base_channels, 4, stride=2, padding=1)

        # Output
        self.output_conv = nn.Sequential(
            nn.GroupNorm(min(32, base_channels), base_channels),
            nn.SiLU(),
            nn.Conv2d(base_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t, context, cfg_scale=1.0):
        t_emb = get_timestep_embedding(t, self.time_emb_dim // 4)
        t_emb = self.time_mlp(t_emb)

        def denoise(x, t_emb, context):
            h = self.input_conv(x)

            # Encoder
            h1 = self.down1(h, t_emb, context)
            h1_cross = self.cross1(h1, context)
            h1_down = self.downsample1(h1_cross)

            h2 = self.down2(h1_down, t_emb, context)
            h2_cross = self.cross2(h2, context)
            h2_down = self.downsample2(h2_cross)

            h3 = self.down3(h2_down, t_emb, context)
            h3_cross = self.cross3(h3, context)
            h3_down = self.downsample3(h3_cross)

            # Middle
            h_mid = self.middle1(h3_down, t_emb, context)
            h_mid = self.middle_attn(h_mid)
            h_mid = self.middle2(h_mid, t_emb, context)

            # Decoder
            h3_cross_resized = F.interpolate(h3_cross, size=h_mid.shape[-2:], mode='nearest')
            h = self.up3(torch.cat([h_mid, h3_cross_resized], dim=1), t_emb, context)
            h = self.upsample3(h)
            h = self.cross_up3(h, context)

            h2_cross_resized = F.interpolate(h2_cross, size=h.shape[-2:], mode='nearest')
            h = self.up2(torch.cat([h, h2_cross_resized], dim=1), t_emb, context)
            h = self.upsample2(h)
            h = self.cross_up2(h, context)

            h1_cross_resized = F.interpolate(h1_cross, size=h.shape[-2:], mode='nearest')
            h = self.up1(torch.cat([h, h1_cross_resized], dim=1), t_emb, context)
            h = self.upsample1(h)

            return self.output_conv(h)

        if cfg_scale == 1.0 or context is None:
            return denoise(x, t_emb, context)

        uncond = denoise(x, t_emb, context=None)
        cond = denoise(x, t_emb, context)
        return uncond + cfg_scale * (cond - uncond)
import torch
from diffusers import AutoencoderKL, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from PIL import Image
import numpy as np
from tqdm import tqdm
import argparse
import sys



def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def generate_images_direct(unet_path="output/KahabMinGenT2Im-v1.pt", device="cuda", output_dir="output", prompt=None):
    """Generate 256x256 images with a custom UNet and user-specified text prompt"""
    seed_everything(42)
    print(f"Using device: {device}")

 
    print("Loading trained UNet...")
    unet = UNetConditional(in_channels=4, base_channels=128, context_dim=768)
    checkpoint = torch.load(unet_path, map_location=device, weights_only=True)
    unet.load_state_dict(checkpoint['model_state_dict'])
    unet = unet.to(device).eval()

    # Create scheduler
    scheduler = DDPMScheduler(num_train_timesteps=1000)

    # Get prompt from user if not provided
    if prompt is None:
        # Check if running in Jupyter
        if 'ipykernel' in sys.modules:
            prompt = input("Enter your text prompt (e.g., 'A friendly dragon'): ").strip()
        else:
            prompt = ""  # Will be handled by argparse default or user input
        if not prompt:
            prompt = "A friendly dragon"  # Default prompt if empty

    test_prompts = [prompt]

    print("🎨 Generating 256x256 images...")
    for i, prompt in enumerate(test_prompts):
        print(f"Generating: {prompt}")
        try:
            with torch.no_grad():
                # Encode prompt
                inputs = tokenizer(
                    prompt,
                    padding="max_length",
                    truncation=True,
                    max_length=77,
                    return_tensors="pt"
                )
                inputs = {k: v.to(device) for k, v in inputs.items()}
                text_embeddings = text_encoder(**inputs).last_hidden_state
                print(f"Text embeddings shape: {text_embeddings.shape}, device: {text_embeddings.device}")

                # Create random latents for 256x256 output (256/8 = 32 due to VAE scaling)
                latents = torch.randn(1, 4, 32, 32, device=device, dtype=torch.float32)
                print(f"Initial latents shape: {latents.shape}, device: {latents.device}")

                # Set timesteps
                scheduler.set_timesteps(500)

                # Denoising loop
                for t in tqdm(scheduler.timesteps, desc=f"Denoising {prompt}"):
                    t_tensor = torch.tensor([t], device=device, dtype=torch.long)
                    noise_pred = unet(latents, t_tensor, context=text_embeddings)
                    latents = scheduler.step(noise_pred, t, latents).prev_sample

                print(f"Final latents shape: {latents.shape}")

                # Decode latents to image
                latents = latents / 0.18215
                images = vae.decode(latents).sample
                images = (images / 2 + 0.5).clamp(0, 1)  # Denormalize
                images = images.cpu().permute(0, 2, 3, 1).numpy()
                image = Image.fromarray((images[0] * 255).astype(np.uint8))

                # Save
                filename = f"{output_dir}/generated_256_.png"
                image.save(filename)
                print(f"✅ Saved: {filename}")

        except Exception as e:
            print(f"❌ Error generating '{prompt}': {e}")
            print(f"Error type: {type(e).__name__}")
            continue

def main():
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        generate_images_direct(
            unet_path="output/KahabMinGenT2Im-v1.pt",
            device="cuda" if torch.cuda.is_available() else "cpu",
            output_dir="output",
            prompt=None
        )
    else:
        parser = argparse.ArgumentParser(description="Generate images with custom UNet and text prompt")
        parser.add_argument("--unet_path", type=str, default="output/KahabMinGenT2Im-v1.pt", help="Path to UNet checkpoint")
        parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu", help="Device to use (cuda or cpu)")
        parser.add_argument("--output_dir", type=str, default="output", help="Output directory for generated images")
        parser.add_argument("--prompt", type=str, default=None, help="Text prompt for image generation")
        args = parser.parse_args()

        generate_images_direct(
            unet_path=args.unet_path,
            device=args.device,
            output_dir=args.output_dir,
            prompt=args.prompt
        )



In [6]:
generate_images_direct(prompt="a dog plyaing")

Using device: cuda
Loading trained UNet...


FileNotFoundError: [Errno 2] No such file or directory: 'output/KahabMinGenT2Im-v1.pt'